In [ ]:
!pip install deeppavlov
!pip install kenlm
!pip install sacremoses

import os
from google.colab import drive
import zipfile
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import glob
from deeppavlov import build_model, configs
import re

drive.mount('/content/gdrive/')
!unzip -q /content/gdrive/My\ Drive/2014.zip -d train

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.2/489.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.1
    Uninstalling tqdm-4.66.1:
      Successfully uninstalled tqdm-4.66.1
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully 

сначала загрузим документы из одного года, архив лежит на гугл-диске

In [ ]:
year = '2014' # valid years 1991-
datasetdir = 'train/' #your path here

#different types of work for different types of data
fieldsid = ['authorByOP',  'docDateByOP',  'docNumberByOP',   'pravogovruNd',  'IssuedByIPS',  'DocDateIPS' ,  'DocNumberIPS']
fieldsop = ['opdate',  'opnumber', 'opweekcode']
fieldshd = ['headingIPS', 'docTitleByOP', 'docTypeByOP']
fieldstxt = ['textOCR',  'textIPS']
fieldsclass = ['classifierByIPS',  'keywordByIPS']
alldata = [] #we will collect dictionaries to this in order to make a Data Frame of it

#iterate through XML files, parse a file into a dictionary and add to alldata
for inputfilename in glob.glob(datasetdir + str(year) + '*.xml'):
    tree = ET.parse(inputfilename)
    root = tree.getroot()
    bufferdict = {}
    bufferdict['fileid'] = inputfilename[-15:]
    for field in fieldsid:
        try:
            bufferdict[field] = root.find("./meta/identification/" + field).get('val')
        except:
            bufferdict[field] = None

    for field in fieldsop:
        try:
            bufferdict[field] = root.find("./meta/publication/").get(field)
        except:
            bufferdict[field] = None

    for field in fieldshd:
        try:
            bufferdict[field] = root.find("./meta/identification/" + field).text
        except:
            bufferdict[field] = None

    for field in fieldstxt:
        try:
            bufferdict[field] = root.find("./body/" + field).text
        except:
            bufferdict[field] = None
    #then since we can have multiple values from classifier and keywords field we have to represent lists of them via semicolon
    kwall = ''
    try:
        kw = root.findall("./meta/keywords/keywordByIPS")
        for i in kw:
            kwall = kwall + ';' + i.get('val')
        if kwall[0] == ';':
            bufferdict['keywordByIPS'] = kwall[1:]
    except:
        bufferdict['keywordByIPS'] = None
    clasall = ''
    try:
        cla = root.findall("./meta/references/classifierByIPS")
        for i in cla:
            clasall = clasall + ';' + i.get('val')
        if clasall[0] == ';':
            bufferdict['classifierByIPS'] = clasall[1:]
    except:
        bufferdict['classifierByIPS'] = None

    alldata.append(bufferdict)

# finally make a dataframe
df = pd.DataFrame(alldata,  columns = ['fileid']+ fieldsid + fieldsop + fieldshd + fieldstxt + fieldsclass)
# print some results:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18034 entries, 0 to 18033
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   fileid           18034 non-null  object
 1   authorByOP       14539 non-null  object
 2   docDateByOP      14539 non-null  object
 3   docNumberByOP    14539 non-null  object
 4   pravogovruNd     8669 non-null   object
 5   IssuedByIPS      8669 non-null   object
 6   DocDateIPS       8669 non-null   object
 7   DocNumberIPS     8669 non-null   object
 8   opdate           14539 non-null  object
 9   opnumber         14539 non-null  object
 10  opweekcode       14539 non-null  object
 11  headingIPS       8669 non-null   object
 12  docTitleByOP     12603 non-null  object
 13  docTypeByOP      14538 non-null  object
 14  textOCR          14538 non-null  object
 15  textIPS          8669 non-null   object
 16  classifierByIPS  8048 non-null   object
 17  keywordByIPS     8046 non-null 

чистка текстов от нанов, символов табуляции

In [ ]:
df_to_work = df.copy()
df_to_work = df_to_work[['docTypeByOP', 'textOCR']]
df_to_work['textOCR'] = df_to_work['textOCR'].str.replace('\n', ' ')
df_to_work['textOCR'] = df_to_work['textOCR'].str.replace('-', ' ')
df_to_work=df_to_work.dropna()
texts = df_to_work['textOCR'].tolist()
df_to_work

,docTypeByOP,textOCR
1,Договор,ДОГОВОР между Российской Федерацией и Республи...
2,Постановление,ПРАВИТЕЛЬСТВО ЗАБАЙКАЛЬСКОГО КРАЯ ПОСТАНОВ...
3,Распоряжение,ПРАВИТЕЛЬСТВО Россшйской ФЕДЕРАЦИИ РАСПОРЯЖ...
5,Постановление,А№1ИСТРАЦИЯ ТАМБОВСКОЙ ОБЛАСТИ ПОСТАНОВЛЕН...
6,Постановление,ПРАВИТЕЛЬСТВО РЯЗАНСКОЙ ОБЛАСТИ ПОСТАНОВЛЕН...
...,...,...
18029,Указ,УКАЗ ПРЕЗИДЕНТА РОССИЙСКОЙ ФЕДЕРАЦИИ Об а...
18030,Постановление,Пр ?! .итепьство Как!!! :… и… !! ! Про… оп...
18031,Распоряжение,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ РАСПОРЯЖ...
18032,Постановление,СМОЛЕНСКАЯ ОБЛАСТНАЯ ДУМА ПОСТАНОВЛЕНИЕ...


в тексте много опечаток и странных символов, идея - найти все эти символы во всех текстах документов и сохранить их в отдельный сет, вывести его и положить символы в функцию, которая почистит текст от них

функция для поиска всех плохих символов - то есть тех, которые не являются русскими буквами, пробелом или точкой

In [ ]:
def find_bad_symb(l1, l2):
  bad_symb = set()
  for i in range(len(l1)):
    bad_symb.update(set(l1[i])-set(l2))
  return bad_symb


russian_letters = [chr(i) for i in range(0x0410, 0x044F+1)]
russian_letters.append(' ')
russian_letters.append('.')
for i in range(0,11):
  russian_letters.append(str(i))

all_bed = set()
for i in texts:
  all_bed.update(find_bad_symb(i, russian_letters))
all_bed

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'B',
 'C',
 'E',
 'F',
 'G',
 'I',
 'L',
 'O',
 'T',
 'U',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '©',
 '«',
 '®',
 '°',
 '»',
 'Ё',
 'ё',
 'і',
 '—',
 '‘',
 '’',
 '‚',
 '“',
 '”',
 '„',
 '…',
 '‹',
 '›',
 '№'}

теперь можно прямо перечислить все эти символы и положить их в функцию, которая очистит от них текст

In [ ]:
def remove_bad_symbols(string):
  bad_symb = re.compile("["
                u"\№"  # emoticons
                u"\і"
                u"\ё"
                u"\{"
                u"\›"
                u"\`"
                u"\“"
                u"\№"
                u"\@"
                u"\—"
                u"\/"
                u"\°"
                u"\«"
                u"\,"
                u"\)"
                u"\…"
                u"\>"
                u"\="
                u"\#"
                u"\:"
                u"\і"
                u"\%"
                u"\]"
                u"\?"
                u"\‘"
                u"\_"
                u"\;"
                u"\["
                u"\*"
                u"\'"
                u"\""
                u"\$"
                u"\<"
                u"\^"
                u"\‚"
                u"\‹"
                u"\’"
                u"\”"
                u"\»"
                u"\&"
                u"\Ё"
                u"\("
                u"\„"
                u"\!"
                u"\|"
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
                 "]+", flags=re.UNICODE)
  return bad_symb.sub(r'', string)


data_test = df_to_work.copy()
data_test['textOCR'] = df_to_work['textOCR'].apply(remove_bad_symbols)
data_test = data_test[['docTypeByOP', 'textOCR']]
data_test

,docTypeByOP,textOCR
1,Договор,ДОГОВОР между Российской Федерацией и Республи...
2,Постановление,ПРАВИТЕЛЬСТВО ЗАБАЙКАЛЬСКОГО КРАЯ ПОСТАНОВ...
3,Распоряжение,ПРАВИТЕЛЬСТВО Россшйской ФЕДЕРАЦИИ РАСПОРЯЖ...
5,Постановление,А1ИСТРАЦИЯ ТАМБОВСКОЙ ОБЛАСТИ ПОСТАНОВЛЕНИ...
6,Постановление,ПРАВИТЕЛЬСТВО РЯЗАНСКОЙ ОБЛАСТИ ПОСТАНОВЛЕН...
...,...,...
18029,Указ,УКАЗ ПРЕЗИДЕНТА РОССИЙСКОЙ ФЕДЕРАЦИИ Об а...
18030,Постановление,Пр .итепьство Как и Про опьна 0 18...
18031,Распоряжение,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ РАСПОРЯЖ...
18032,Постановление,СМОЛЕНСКАЯ ОБЛАСТНАЯ ДУМА ПОСТАНОВЛЕНИЕ...


в текстах все равно сохранились опечатки в некоторых словах. Для того, чтобы исправить хотя бы одну ошибку в слове, прогоним текст через модель от DeepPavlov - spelling corrector на основе метрики Левенштейна

In [ ]:
model = build_model('levenshtein_corrector_ru', download=True)

def spelling_correction(col):
  return model([col])[0]



2023-12-15 05:35:41.121 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/deeppavlov_data/vocabs/russian_words_vocab.dict.gz to /root/.deeppavlov/downloads/russian_words_vocab.dict.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/deeppavlov_data/vocabs/russian_words_vocab.dict.gz to /root/.deeppavlov/downloads/russian_words_vocab.dict.gz
100%|██████████| 10.7M/10.7M [00:01<00:00, 7.48MB/s]
2023-12-15 05:35:43.603 INFO in 'deeppavlov.core.data.utils'['utils'] at line 276: Extracting /root/.deeppavlov/downloads/russian_words_vocab.dict.gz archive into /root/.deeppavlov/downloads/vocabs
INFO:deeppavlov.core.data.utils:Extracting /root/.deeppavlov/downloads/russian_words_vocab.dict.gz archive into /root/.deeppavlov/downloads/vocabs
2023-12-15 05:35:44.827 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/lang_models/ru_wiyalen_no_punkt.arpa.binary.gz to /roo

In [ ]:
df_corrected = data_test.copy()
df_corrected = df_corrected[df_corrected['textOCR'].str.len()<5000]
df_corrected['textOCR'] = df_corrected['textOCR'].apply(spelling_correction)
df_corrected

,docTypeByOP,textOCR
1,Договор,договор между российской федерацией и республи...
2,Постановление,правительство забайкальского края постановлени...
3,Распоряжение,правительство российской федерации распоряжени...
5,Постановление,а1истрация тамбовской области постановление ек...
6,Постановление,правительство рязанской области постановление ...
...,...,...
18029,Указ,указ президента российской федерации об атаман...
18030,Постановление,пр .итепьство как и про опьна 0 1801000 04 1 п...
18031,Распоряжение,правительство российской федерации распоряжени...
18032,Постановление,смоленская областная дума постановление от йй7...


In [ ]:
df_corrected.to_csv('//content/gdrive/MyDrive/dataset_2014_corrected.csv', index=False)